In [32]:
initial_state = [ [ 1, 2, 3 ],  
            [ 5, 6, 0 ],  
            [ 7, 8, 4 ] ]  

final_state = [ [ 1, 2, 3 ],  
        [ 5, 8, 6 ],  
        [ 0, 7, 4 ] ]  

# 0 is used to represent the blank tile

In [33]:
def print_puzzle_state(state):
    for row in state:
        print("-----------------------------")
        for tile in row:
            print("|   {}   |".format(tile),end =" ")
        print()  # Move to the next line after each row
    print("-----------------------------\n\n")



def find_zero_index(state):
    for i in range(3):
        for j in range(3):
            if(state[i][j] == 0):
                return (i,j)

# MOVE BLANK LEFT - 1
# MOVE BLANK RIGHT - 2
# MOVE BLANK UP - 3
# MOVE BLANK DOWN - 4

def possible_moves(i,j):
    # (i,j) is the index of zero(blank) tile
    actions_domain = [1,2,3,4]
    
    if i==0:
        #remove up
        actions_domain.remove(3)
    elif i==2:
        #remove down
        actions_domain.remove(4)
    
    if j==0:
        #remove left
        actions_domain.remove(1)
    elif j==2:
        #remove right
        actions_domain.remove(2)
    
    return actions_domain
    
    
    
    
import copy
# MOVE BLANK LEFT - 1
# MOVE BLANK RIGHT - 2
# MOVE BLANK UP - 3
# MOVE BLANK DOWN - 4

def possible_state(state,move,i,j):
    state_copy = copy.deepcopy(state)
    if move == 1:
        #left
        state_copy[i][j] , state_copy[i][j-1] = state_copy[i][j-1] , state_copy[i][j]
        
    elif move == 2:
        #right
        state_copy[i][j] , state_copy[i][j+1] = state_copy[i][j+1] , state_copy[i][j]
        
    elif move == 3:
        #up
        state_copy[i][j] , state_copy[i-1][j] = state_copy[i-1][j] , state_copy[i][j]
    
    elif move == 4:
        #down
        state_copy[i][j] , state_copy[i+1][j] = state_copy[i+1][j] , state_copy[i][j]
        
    return state_copy
    

In [34]:
i,j = find_zero_index(initial_state)
my_list = []
for move in possible_moves(i,j):
    my_list.append(possible_state(initial_state,move,i,j))
    
print_puzzle_state(my_list[0])
print_puzzle_state(my_list[1])
print_puzzle_state(my_list[2])

-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   0   | |   6   | 
-----------------------------
|   7   | |   8   | |   4   | 
-----------------------------


-----------------------------
|   1   | |   2   | |   0   | 
-----------------------------
|   5   | |   6   | |   3   | 
-----------------------------
|   7   | |   8   | |   4   | 
-----------------------------


-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   6   | |   4   | 
-----------------------------
|   7   | |   8   | |   0   | 
-----------------------------




In [35]:
def heuristic(state , goal):
    #RETURNS THE NUMBER OF MISPLACED TILES
    h = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != goal[i][j]:
                h = h+1
                
    return h

In [36]:
print(heuristic(initial_state , final_state))

4


In [37]:
def check_unique(OPEN , CLOSED , new_state):
    my_list = list(OPEN) + CLOSED #merge both lists
    for (child,parent,depth) in my_list:
        if set(tuple(element) for element in child) == set(tuple(element) for element in new_state):
            return False
        
    return True

def sort_by_third(list_of_tuples):
    """Sorts a list of tuples by the third element in each tuple.

    Args:
        list_of_tuples: The list to be sorted.

    Returns:
        A new list with the tuples sorted in ascending order by the third element.
    """

    return sorted(list_of_tuples, key=lambda x: x[2])  # Sort based on third element (index 2)


In [38]:
def get_parent(state,LIST):
    for (c,p,d) in LIST:
        if state == c:
            return p
        

def create_path(goal_state,CLOSED):
    my_path = [goal_state]
    
    p = get_parent(goal_state,CLOSED)
    while p != None:  #till we get to initial_state
        my_path.append(p)
        p = get_parent(p,CLOSED)
        
    return my_path

In [39]:
OPEN = [] # used as a priority queue
CLOSED = []

def best_first_search(initial_state,goal_state):
    OPEN = []
    CLOSED = []
    h = heuristic(initial_state , goal_state)
    OPEN.append((initial_state,None,h))   # searchNode=(currentState,parentState, heuristicValue)
    
    while OPEN:
        OPEN = sort_by_third(OPEN)  #FOR PRIORITY QUEUE
        state_pair = OPEN.pop(0)
        CLOSED.append(state_pair)
        
        state = state_pair[0] #child node only
        h = state_pair[2] #heuristic value of current state
        
        if state == goal_state:
            print("Goal state reached.")
            #print the path taken
            my_path = create_path(goal_state,CLOSED)
            for s in my_path:
                print_puzzle_state(s)
            return True
        
        (i,j) = find_zero_index(state)  #stores the index of xero index in the current state
        for move in possible_moves(i,j):
            new_state = possible_state(state,move,i,j)
            if check_unique(OPEN , CLOSED , new_state):
                new_h = heuristic(new_state , goal_state)
                OPEN.append((new_state,state,new_h))
                
    print("Goal state not found")
    return False

In [40]:
initial_state = [ [ 1, 2, 3 ],  
            [ 5, 6, 0 ],  
            [ 7, 8, 4 ] ]  

final_state = [ [ 1, 2, 3 ],  
        [ 5, 8, 6 ],  
        [ 0, 7, 4 ] ] 

In [41]:
best_first_search(initial_state,final_state)

Goal state reached.
-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   8   | |   6   | 
-----------------------------
|   0   | |   7   | |   4   | 
-----------------------------


-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   8   | |   6   | 
-----------------------------
|   7   | |   0   | |   4   | 
-----------------------------


-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   0   | |   6   | 
-----------------------------
|   7   | |   8   | |   4   | 
-----------------------------


-----------------------------
|   1   | |   2   | |   3   | 
-----------------------------
|   5   | |   6   | |   0   | 
-----------------------------
|   7   | |   8   | |   4   | 
-----------------------------




True